# Rešenje zadatka

In [335]:
import cv2
import numpy as np

### Pretprocesiranje slike
Slika se prevodi u **greyscale**. 

Zatim se na nju primenjuje **median blur** sa ciljem smanjenja tačkastog (salt and peper) šuma iz pozadine slike. 
> The function cv2.medianBlur() takes median of all the pixels under kernel area and central element is replaced with this median value. This is highly effective against salt-and-pepper noise in the images. 

Primena median blura za rezultat ima da se smanji dimenzija tačkica, te da postanu tamnije.

Primenjuje **otvaranje**. Otvaranje u celosti ukloni tačkice sa slike. Primena otvaranja, bez median blura rezultovala je značajnim oštećenjem brojeva.

Na kraju, primenjuje se globalni **treshold**.

In [336]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
def processFrame(image):
    #prevođenje slike u greyscale
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)     
    #blur
    image = cv2.medianBlur(image,3)
    #otvaranje = erozija + dilacija
    image = cv2.erode(image, kernel, iterations = 1)
    image = cv2.dilate(image, kernel, iterations = 1)
    #treshold  
    ret, image = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY)
    return image

### Obrada konture broja
Radi bolje tačnosti, kontura na kojoj je pronađen broj se dodatno obrađuje. Skidaju se crni pikseli okvira.
Zatim se veličina slike menja na 28x28 piksela u skladu sa očekivanim ulazom.

In [337]:
def processRegion(region, tolerance = 0):
    mask = region > tolerance
    region = region[np.ix_(mask.any(1),mask.any(0))]            
    return cv2.resize(region,(28,28), interpolation = cv2.INTER_AREA)

### Obrada prvog frejma video snimka

In [338]:
vidcap = cv2.VideoCapture('data/videos/video-3.avi')
#učitaj prvu sliku
success,image = vidcap.read()
#učitaj sliku nakon prve
success,image = vidcap.read()

### Obrada ostalih frejmova video snimka

In [339]:
count = 0
region_count = 0
while count < 10:
    #obradi prethodnu sliku    
    processed_image = processFrame(image); 
    processed_image, contours, hierarchy = cv2.findContours(processed_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if h >= 15 and h <= 20:
            region = processed_image[y:y+h+1,x:x+w+1] 
            region = processRegion(region)
            cv2.imwrite("area_region%d.jpg" % region_count, region)             
            region_count += 1
    #učitaj sledeću sliku
    success,image = vidcap.read()
    count += 1;
vidcap.release()